<a href="https://colab.research.google.com/github/michaelknorr/CHE4076/blob/main/CHE4076_LYS_pH_7_1_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load the Excel file with the sheet name 'Sphere1'
df = pd.read_excel('/content/drive/My Drive/Conc Profile LYS pH 7.1.xls', sheet_name='Sphere1', header=None, usecols="C:S", skiprows=3, nrows=736)
df1=pd.read_excel('/content/drive/My Drive/Conc Profile LYS pH 7.1.xls', sheet_name='Sphere1', header=None, usecols="Y:AO", skiprows=100, nrows=1)
df2 = pd.read_excel('/content/drive/My Drive/Conc Profile LYS pH 7.1.xls', sheet_name='Sphere1', header=None, skiprows=66, usecols="X", nrows=21)
raw_data_array = df.to_numpy()
radial_positions = df1.to_numpy().flatten()  # Flattening the array to make it 1D
time_steps = df2.to_numpy().flatten()  # Rows 102 to 121 (index 101 to 120), column X (index 23)

print(radial_positions.shape)
print(time_steps.shape)

(17,)
(21,)


In [ ]:
import numpy as np

# Number of radial angles per time step
radial_angles_per_timestep = 32

# Calculate the number of time steps
num_time_steps = raw_data_array.shape[0] // radial_angles_per_timestep

# Initialize an array to store the averages
average_raw_data_array = np.zeros((num_time_steps, raw_data_array.shape[1]))

# Calculate the averages for each column (radial position) across every 32 entries (radial angles per time step)
for col in range(raw_data_array.shape[1]):
    for t in range(num_time_steps):
        # Extract the relevant 32 values for this column and time step
        radial_angles = raw_data_array[t * radial_angles_per_timestep:(t + 1) * radial_angles_per_timestep, col]
        # Calculate the average of the 32 values
        average_raw_data_array[t, col] = np.mean(radial_angles)

# Display the shape of the resulting average array
print(f"Average raw data array shape: {average_raw_data_array.shape}")

# Remove the last two rows from the average_raw_data_array to create the trimmed array
average_raw_data_array_trimmed = average_raw_data_array[:-2, :]

# Display the shape of the trimmed array
print(f"Trimmed average raw data array shape: {average_raw_data_array_trimmed.shape}")





Average raw data array shape: (23, 17)
Trimmed average raw data array shape: (21, 17)


In [ ]:
# Find the global minimum and maximum values in the entire trimmed average array
global_min = np.min(average_raw_data_array_trimmed)  # Minimum value in the entire array
global_max = 67.708375
# np.max(average_raw_data_array_trimmed)  # Maximum value in the entire array

# Normalize the data using the global min and max values for the entire array
normalized_data_global = (average_raw_data_array_trimmed - global_min) / (global_max - global_min)


# Display the shape of the normalized data
print(f"Normalized data (global) shape: {normalized_data_global.shape}")

# Optional: display the normalized data
#print(normalized_data_global)



Normalized data (global) shape: (21, 17)


In [ ]:
# Remove rows 0 and 1 from the normalized data
rescaled_normalized_averages = normalized_data_global[:, 1:]

# Display the shape of the rescaled normalized averages
print(f"Rescaled normalized averages shape: {rescaled_normalized_averages.shape}")

# Optional: display the rescaled normalized averages
#print(rescaled_normalized_averages)


Rescaled normalized averages shape: (21, 16)


In [ ]:
pi=np.pi

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Define the diffusion model
def diffusion_model(r, t, t_0, D_e, term=1):
    rs = 0.00075  # cm
    return (((-1)**term) / term) * np.sin(term * np.pi * r * 10**(-4) / rs) * np.exp(-D_e * term**2 * np.pi**2 * (t + t_0) / rs**2)




In [ ]:
from scipy.optimize import curve_fit
import numpy as np

# Define the diffusion model function
def diffusion_model(r, t, t_0, D_e, C_0):
    rs = 0.00075  # cm
    sum_terms = 0

    for term in range(1, 6):
        # Numerically stable exponential term
        exp_term = np.exp(
            np.clip(-D_e * term**2 * np.pi**2 * (t + t_0) / rs**2, -50, 50)
        )
        sum_terms += (((-1) ** term) / term) * np.sin(term * np.pi * r * 10**-4 / rs) * exp_term

    # Avoid division by zero
    r_safe = np.where(r == 0, 1e-10, r)

    return (1 + (2 * rs / (np.pi * r_safe * 10**-4)) * sum_terms) * C_0 + 5.18E-5 * t

# Fit the diffusion model
def fit_diffusion_model(radial_positions, time_steps, normalized_data_global):
    def model_func(r, t_0, D_e, C_0):
        return diffusion_model(r, t, t_0, D_e, C_0)

    initial_guess = [192.97502, 1.3e-10, 0.9184378]
    bounds = ([0, 0, 0], [500, 1e-9, 10])  # Adjust bounds for stability

    params_list = []
    for time_idx, t in enumerate(time_steps):
        r_values = radial_positions
        data_values = normalized_data_global[time_idx, :]

        # Perform curve fitting with bounds
        try:
            params_opt, params_cov = curve_fit(
                model_func, r_values, data_values, p0=initial_guess, bounds=bounds
            )
        except RuntimeError:
            print(f"Fit failed at time index {time_idx}")
            params_opt = [np.nan, np.nan, np.nan]

        params_list.append(params_opt)

    return np.array(params_list)

# Assuming you have the radial_positions, time_steps, and data arrays ready
params = fit_diffusion_model(radial_positions, time_steps, normalized_data_global)
print(params)


[[3.30091192e+02 1.13355550e-10 4.90996939e-01]
 [2.84790160e+02 1.65368611e-10 4.68252166e-01]
 [3.00997270e+02 1.49453921e-10 4.75635342e-01]
 [2.94726727e+02 1.58924073e-10 4.85816557e-01]
 [2.85901764e+02 1.43508125e-10 4.92079547e-01]
 [3.58406404e+02 1.17768800e-10 5.14474216e-01]
 [2.65797981e+02 2.40091741e-10 5.09519684e-01]
 [2.70505112e+02 2.68230703e-10 5.17606355e-01]
 [4.65984975e+02 8.86309964e-10 5.30801925e-01]
 [4.77203065e+02 8.00653394e-10 5.45074582e-01]
 [3.00042794e+02 9.99999998e-10 5.71046509e-01]
 [4.62108181e+02 9.01294989e-10 5.93906868e-01]
 [3.66400227e+02 7.15550331e-10 5.73704248e-01]
 [3.53234706e+02 8.29995664e-10 6.09119185e-01]
 [3.86795821e+02 1.00000000e-09 6.42137583e-01]
 [3.86635633e+02 9.62047890e-10 6.68109697e-01]
 [3.86638743e+02 1.00000000e-09 6.90133631e-01]
 [3.82296858e+02 9.88502131e-10 7.13813832e-01]
 [3.78038050e+02 1.00000000e-09 7.05337669e-01]
 [3.65639589e+02 1.00000000e-09 7.01629936e-01]
 [3.60829597e+02 1.00000000e-09 7.002858